In [ ]:
from google.colab import files
uploaded = files.upload()

Saving fake_and_real_news.csv to fake_and_real_news.csv


In [ ]:
# About the Dataset
# 1. title: the title of a news article
# 2. label: a label that marks whether the news article is real or fake:
#            Fake: Fake news
#            Real: real News

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import os

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
file_path=os.path.abspath('fake_and_real_news.csv')
print(file_path)

/content/fake_and_real_news.csv


In [ ]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('/content/fake_and_real_news.csv')

In [ ]:
news_dataset.shape

(9900, 2)

In [ ]:
# print the first 5 rows of the dataframe
news_dataset.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [ ]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

,0
Text,0
label,0


In [ ]:
# replacing the null values with empty string
_dataset = news_dataset.fillna('')

In [ ]:
# select the Text as the content
news_dataset['content'] = news_dataset['Text']

In [ ]:
print(news_dataset['content'])

0        Top Trump Surrogate BRUTALLY Stabs Him In The...
1       U.S. conservative leader optimistic of common ...
2       Trump proposes U.S. tax overhaul, stirs concer...
3        Court Forces Ohio To Allow Millions Of Illega...
4       Democrats say Trump agrees to work on immigrat...
                              ...                        
9895     Wikileaks Admits To Screwing Up IMMENSELY Wit...
9896    Trump consults Republican senators on Fed chie...
9897    Trump lawyers say judge lacks jurisdiction for...
9898     WATCH: Right-Wing Pastor Falsely Credits Trum...
9899     Sean Spicer HILARIOUSLY Branded As Chickensh*...
Name: content, Length: 9900, dtype: object


In [ ]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [ ]:
print(X)
print(Y)

                                                   Text
0      Top Trump Surrogate BRUTALLY Stabs Him In The...
1     U.S. conservative leader optimistic of common ...
2     Trump proposes U.S. tax overhaul, stirs concer...
3      Court Forces Ohio To Allow Millions Of Illega...
4     Democrats say Trump agrees to work on immigrat...
...                                                 ...
9895   Wikileaks Admits To Screwing Up IMMENSELY Wit...
9896  Trump consults Republican senators on Fed chie...
9897  Trump lawyers say judge lacks jurisdiction for...
9898   WATCH: Right-Wing Pastor Falsely Credits Trum...
9899   Sean Spicer HILARIOUSLY Branded As Chickensh*...

[9900 rows x 1 columns]
0       Fake
1       Real
2       Real
3       Fake
4       Real
        ... 
9895    Fake
9896    Real
9897    Real
9898    Fake
9899    Fake
Name: label, Length: 9900, dtype: object


In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

KeyboardInterrupt: 

In [ ]:
print(news_dataset['content'])

0        Top Trump Surrogate BRUTALLY Stabs Him In The...
1       U.S. conservative leader optimistic of common ...
2       Trump proposes U.S. tax overhaul, stirs concer...
3        Court Forces Ohio To Allow Millions Of Illega...
4       Democrats say Trump agrees to work on immigrat...
                              ...                        
9895     Wikileaks Admits To Screwing Up IMMENSELY Wit...
9896    Trump consults Republican senators on Fed chie...
9897    Trump lawyers say judge lacks jurisdiction for...
9898     WATCH: Right-Wing Pastor Falsely Credits Trum...
9899     Sean Spicer HILARIOUSLY Branded As Chickensh*...
Name: content, Length: 9900, dtype: object


In [ ]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [ ]:
print(X)

[' Top Trump Surrogate BRUTALLY Stabs Him In The Back: ‘He’s Pathetic’ (VIDEO) It s looking as though Republican presidential candidate Donald Trump is losing support even from within his own ranks. You know things are getting bad when even your top surrogates start turning against you, which is exactly what just happened on Fox News when Newt Gingrich called Trump  pathetic. Gingrich knows that Trump needs to keep his focus on Hillary Clinton if he even remotely wants to have a chance at defeating her. However, Trump has hurt feelings because many Republicans don t support his sexual assault against women have turned against him, including House Speaker Paul Ryan (R-WI). So, that has made Trump lash out as his own party.Gingrich said on Fox News: Look, first of all, let me just say about Trump, who I admire and I ve tried to help as much as I can. There s a big Trump and a little Trump. The little Trump is frankly pathetic. I mean, he s mad over not getting a phone call? Trump s refer

In [ ]:
Y.shape

(9900,)

In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 4157)	0.022595178512476097
  (0, 4516)	0.04805565447838848
  (0, 4616)	0.11225667081941841
  (0, 4840)	0.025737391221857323
  (0, 4858)	0.09951101615141389
  (0, 5302)	0.026041351687630762
  (0, 5444)	0.026781075708531513
  (0, 5789)	0.04635498534862242
  (0, 6080)	0.07103081090734875
  (0, 6357)	0.021469028075402484
  (0, 6542)	0.07517839739693684
  (0, 6660)	0.06598949552771385
  (0, 6780)	0.04101782310792941
  (0, 7267)	0.036310004404511806
  (0, 7333)	0.050795609570223575
  (0, 7893)	0.019371546427704948
  (0, 7962)	0.08793377766191733
  (0, 8507)	0.047987020107693575
  (0, 8797)	0.06422983007774563
  (0, 10048)	0.0939210186223511
  (0, 10394)	0.02085198627012458
  (0, 10473)	0.019066377865072256
  (0, 10647)	0.09532376412596097
  (0, 10654)	0.03650602685217999
  (0, 10734)	0.027899357773411775
  :	:
  (9899, 56305)	0.022378135432742263
  (9899, 56362)	0.009212771808343225
  (9899, 56407)	0.0062005898371733726
  (9899, 56571)	0.009234457664346515
  (9899, 56666)	0.08655615447

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

In [ ]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9949494949494949


In [ ]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9863636363636363


In [ ]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]=='Real'):
  print('The news is Real')
else:
  print('The news is Fake')

['Real']
The news is Real


In [ ]:
print(Y_test[2])

Fake
